In [ ]:
from Bio.Cluster import kcluster
from collections import Counter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import subprocess
import shutil
import glob
import time
import os
import re

### Analysis input
input = "TEVp-240412"
input_dataframe = pd.read_csv(f"output/{input}/opt_binders/metrics.csv")

os.makedirs(f"output/{input}/filtered_sequences/filtered_binders", exist_ok=True)
input_dataframe.describe()

In [ ]:
# Functions
# add scaffold name column
def add_scaffold_name_column(filtered, prefix):
    filtered["scaffold_name"] = ""

    for index, row in filtered.iterrows():
        path = row["model_path"]
        prefix = prefix
        file_name = path.split("/")[-1]
        parts = file_name.split(prefix)[-1].split("_")

        if len(parts) >= 5:
            result = f"{parts[0]}_{parts[1]}"
        else:
            result = parts[0].split(".")[0]

        filtered.at[index, "scaffold_name"] = result

    return filtered


def repeat_rows_by_column_value(df, column_name, number):
    unique_values = df[column_name].unique()
    repeated_rows = []

    for value in unique_values:
        subset = df[df[column_name] == value]
        num_repeats = min(number, subset.shape[0])
        repeated_rows.extend([subset.iloc[i, :] for i in range(num_repeats)])

    repeated_df = pd.DataFrame(repeated_rows)
    return repeated_df

## Filter dataframe
tipical metrics:
- plddt > 0.85-0.9
- i_pae < 5

additional metrics:
- charge < -2

In [ ]:
filtered = input_dataframe[
    (input_dataframe["plddt"] > 0.90)
    & (input_dataframe["i_pae"] < 7)
    & (input_dataframe["ddg"] < -50)
    & (input_dataframe["charge"] < -2)
    & (input_dataframe["ddg_dsasa_100"] < -2.5)
    &
    # (input_dataframe["ddgscore_dsasa_100"]<-2.5)&
    (input_dataframe["shape_comp"] > 0.5)
    & (input_dataframe["vbuns_int"] <= 1)
    & (input_dataframe["cms"] > 500)
    & (input_dataframe["hyd_contacts"] > 4)
]
# (input_dataframe["sap"]<100)]#&
# (input_dataframe["dG"]<30)]

filtered = filtered.sort_values(by="ddg_dsasa_100", ascending=True).drop_duplicates(
    "seq"
)
filtered.to_csv(
    f"output/{input}/filtered_sequences/0_filtered_binders.csv", index=False
)
filtered.reset_index(inplace=True)
filtered

In [ ]:
filtered["rfd_id"] = filtered["input_pdb"].apply(
    lambda x: x.split(input + "_")[-1].split(".pdb")[0]
)
filtered["rfd_id"]


rfd_id_counts = filtered["rfd_id"].value_counts()

# Get the number of unique 'rfd_id' values
unique_rfd_ids_count = filtered["rfd_id"].nunique()

# Display the counts of each 'rfd_id' and the number of unique 'rfd_id' values
print("Counts of each 'rfd_id':")
print(rfd_id_counts)
print("\nNumber of unique 'rfd_id' values:", unique_rfd_ids_count)

In [ ]:
filtered = add_scaffold_name_column(filtered, input + "_")
filtered

In [ ]:
# Calculate statistics on scaffolds
scaffold_counts = filtered["scaffold_name"].value_counts()
total_unique_scaffolds = len(scaffold_counts)
total_scaffold_instances = scaffold_counts.sum()

print("Total unique scaffolds:", total_unique_scaffolds)
print("Total scaffold instances:", total_scaffold_instances)
print("\nScaffold counts:")
print(scaffold_counts)

In [ ]:
def repeat_rows_by_column_value(df, column_name, number):
    unique_values = df[column_name].unique()
    repeated_rows = []

    for value in unique_values:
        subset = df[df[column_name] == value]
        num_repeats = min(number, subset.shape[0])
        repeated_rows.extend([subset.iloc[i, :] for i in range(num_repeats)])

    repeated_df = pd.DataFrame(repeated_rows)
    return repeated_df


filtered = repeat_rows_by_column_value(filtered, "rfd_id", 200)
filtered.to_csv(
    f"output/{input}/opt_binders/filtered_af2_rf2.csv"
)
filtered

## Test with ColabFold and Rosettafold2

In [ ]:
def generate_fasta_sequence_from_pdb(pdb_file, output_folder=None):
    ca_pattern = re.compile(
        "^ATOM\s{2,6}\d{1,5}\s{2}CA\s[\sA]([A-Z]{3})\s([\s\w])|^HETATM\s{0,4}\d{1,5}\s{2}CA\s[\sA](MSE)\s([\s\w])"
    )
    aa3to1 = {
        "ALA": "A",
        "VAL": "V",
        "PHE": "F",
        "PRO": "P",
        "MET": "M",
        "ILE": "I",
        "LEU": "L",
        "ASP": "D",
        "GLU": "E",
        "LYS": "K",
        "ARG": "R",
        "SER": "S",
        "THR": "T",
        "TYR": "Y",
        "HIS": "H",
        "CYS": "C",
        "ASN": "N",
        "GLN": "Q",
        "TRP": "W",
        "GLY": "G",
        "MSE": "M",
    }
    filename = os.path.basename(pdb_file).split(".")[0]
    chain_dict = dict()
    chain_list = []

    with open(pdb_file, "r") as fp:
        for line in fp:
            if line.startswith("ENDMDL"):
                break
            match_list = ca_pattern.findall(line)
            if match_list:
                resn = match_list[0][0] + match_list[0][2]
                chain = match_list[0][1] + match_list[0][3]
                if chain in chain_dict:
                    chain_dict[chain] += aa3to1[resn]
                else:
                    chain_dict[chain] = aa3to1[resn]
                    chain_list.append(chain)

    fasta_sequence = f">{filename}\n"
    for i, chain in enumerate(chain_list):
        fasta_sequence += chain_dict[chain]
        if i < len(chain_list) - 1:
            fasta_sequence += ":"

    if output_folder:
        output_file = os.path.join(output_folder, f"{filename}.fasta")
        with open(output_file, "w") as fp:
            fp.write(fasta_sequence)

    return chain_dict


def process_a3m_file_rf2(
    binder_sequence, sample_a3m_path, processed_file_name, verbose=False
):
    if os.path.isfile(sample_a3m_path):
        with open(sample_a3m_path, "r") as file:
            lines = file.readlines()

        # Replace old_binder_sequence with binder_sequence in the rest of the lines
        if verbose:
            print(lines[1])
        old_binder_sequence = lines[1].strip().split("/")[1]
        if verbose:
            print(old_binder_sequence)
        for i in range(0, len(lines)):
            lines[i] = lines[i].replace(old_binder_sequence, binder_sequence)

            # Check if the line contains "-" and replace it accordingly
            if not lines[i].startswith(">") and lines[i].split("/")[1].startswith("-"):
                lines[i] = (
                    lines[i].split("/")[0] + "/" + "-" * len(binder_sequence) + "\n"
                )

        # Save the processed lines to a new file
        new_file_path = f"{processed_file_name}"
        with open(new_file_path, "w") as new_file:
            new_file.writelines(lines)


def process_a3m_file_af2(binder_sequence, sample_a3m_path, processed_file_name):
    if os.path.isfile(sample_a3m_path):
        with open(sample_a3m_path, "r") as file:
            lines = file.readlines()

        # Extract the target sequence from the first line
        target_sequence_line = lines[0].strip()
        if target_sequence_line.startswith("#"):
            target_length = int(target_sequence_line[1:].split(",")[0].strip())
            # print(target_length)
        else:
            raise ValueError(
                "The target sequence is not properly specified in the first line."
            )

        # Change line 0
        lines[0] = f"#{target_length},{len(binder_sequence)}\t1,1\n"
        # Change line 3
        target_sequence = lines[2].strip()[:target_length]
        old_binder_sequence = lines[2].strip()[target_length:]
        lines[2] = target_sequence + binder_sequence + "\n"

        # Replace old_binder_sequence with binder_sequence in the rest of the lines
        for i in range(3, len(lines)):
            lines[i] = lines[i].replace(old_binder_sequence, binder_sequence)

        # Write the processed lines to the output file
        with open(processed_file_name, "w") as processed_file:
            processed_file.writelines(lines)

### Generate MSA files

In [ ]:
# Create folders for samples
# make from whole sequence
msa_folder_rf2 = f"output/{input}/filtered_sequences/msa_inputs_rf2"
os.makedirs(msa_folder_rf2, exist_ok=True)

sample_msa_folder_rf2 = f"{msa_folder_rf2}/sample"
os.makedirs(sample_msa_folder_rf2, exist_ok=True)

# AF2
msa_folder_af2 = f"output/{input}/filtered_sequences/msa_inputs_af2"
os.makedirs(msa_folder_rf2, exist_ok=True)

sample_msa_folder_af2 = f"{msa_folder_af2}/sample"
os.makedirs(sample_msa_folder_af2, exist_ok=True)

# ?
filtered_binders_path = f"output/{input}/filtered_sequences/filtered_binders"  # ??
os.makedirs(filtered_binders_path, exist_ok=True)

# copy pdbs to filtered binders
for i, row in filtered.iterrows():
    # Copy pdb in filtered folder
    model_path = row["model_path"]
    model_name = model_path.split("/")[-1]
    new_model_path = f"{filtered_binders_path}/{model_name}"
    shutil.copyfile(model_path, new_model_path)

In [ ]:
filtered["seq"][0].split("/")[0]

In [ ]:
"""
If you performed RFdiffusion with partial protein sequence,
here you can provide the whole sequence of the target protein.
"""

# same as the sequence for RFdiffusion
target_whole_seq = filtered["seq"][0].split("/")[0]
# or provide the whole sequence of the target protein
# target_whole_seq="MNKSQLYPDSPLTDQDFNQLDQTVIEAARRQLVGRRFIELYGPLGRGMQSVFNDIFMESHEAKMDFQGSFDTEVESSRRVNYTIPMLYKDFVLYWRDLEQSKALDIPIDFSVAANAARDVAFLEDQMIFHGSKEFDIPGLMNVKGRLTHLIGNWYESGNAFQDIVEARNKLLEMNHNGPYALVLSPELYSLLHRVHKDTNVLEIEHVRELITAGVFQSPVLKGKSGVIVNTGRNNLDLAISEDFETAYLGEEGMNHPFRVYETVVLRIKRPAAICTLIDPEE"


sample_a3m_path_rf2 = f"{msa_folder_rf2}/sample/sample.a3m"
sample_a3m_path_af2 = f"{msa_folder_af2}/sample/sample.a3m"


random_binder_seq = filtered["seq"][0].split("/")[-1]
print(f"binder sequence: {random_binder_seq}")
print(f"target sequence: {target_whole_seq}")
# Create fasta
fasta_file_af2 = f"{sample_msa_folder_af2}/fasta.fasta"
with open(f"{fasta_file_af2}", "w") as f:
    # Write the sequence to the file in FASTA format
    f.write(f">sample\n{target_whole_seq}:{random_binder_seq}\n")

In [ ]:
### Generate MSA for AF2 and RF2


### AF2
input_pdb = filtered["model_path"][0]
slurm_args = ""  # "--output=/dev/null"
cmd = f"sbatch {slurm_args} helper_scripts/get_msa_colabfold.sh {input_pdb} {sample_msa_folder_af2}"

# Run the system command to execute the get_msa.py script
# a3m_path_af2 = f"{sample_msa_folder_af2}/{input_pdb.split('/')[-1].split('.')[0]}.a3m"
if not os.path.exists(sample_a3m_path_af2):
    os.system(cmd)

### RF2
cmd_rf2 = f"sbatch {slurm_args} helper_scripts/get_msa_rf.sh {fasta_file_af2} {sample_msa_folder_rf2}"

# Run the system command to execute the get_msa.py script
# sample_a3m_path = f"{sample_msa_folder_rf2}/sample.a3m"
if not os.path.exists(sample_a3m_path_rf2):
    os.system(cmd_rf2)


### WAITING
# Wait until MSA is calculated...
while not os.path.exists(sample_a3m_path_af2):
    time.sleep(1)

print("MSA calculation for AF2 is complete.")

while not os.path.exists(sample_a3m_path_rf2):
    time.sleep(1)
print("MSA calculation for RF2 is complete.")

In [ ]:
### Generate other MSA files for RF2 and copy binders

filtered["new_model_path"] = None
filtered["generated_a3m_rf2"] = None

filtered_binders_path = f"output/{input}/filtered_sequences/filtered_binders"
os.makedirs(filtered_binders_path, exist_ok=True)

binder_chain = "B"

for i, row in filtered.iterrows():
    # Copy pdb in filtered folder
    model_path = row["model_path"]
    model_name = model_path.split("/")[-1]
    new_model_path = f"{filtered_binders_path}/{model_name}"
    # shutil.copyfile(model_path, new_model_path)
    binder_seq = row["seq"].split("/")[-1]

    # binder_seq=generate_fasta_sequence_from_pdb(row["model_path"])["B"]
    a3m_file_name = f"{msa_folder_rf2}/{model_name.split('.')[0]}.a3m"
    sample_a3m_path = sample_a3m_path_rf2
    process_a3m_file_rf2(binder_seq, sample_a3m_path, a3m_file_name)

    # Update DataFrame with new values
    filtered.loc[i, "new_model_path"] = new_model_path
    filtered.loc[i, "generated_a3m_rf2"] = a3m_file_name

# Check A3M files generated
msa_a3m_inputs = glob.glob(f"{msa_folder_rf2}/*a3m")
print(f"There is {len(msa_a3m_inputs)} a3m files ready to be predicted with RF2!")

### Generate other MSA files for AF2 and copy binders

filtered["new_model_path"] = None
filtered["generated_a3m_af2"] = None


binder_chain = "B"

for i, row in filtered.iterrows():
    # Copy pdb in filtered folder
    model_path = row["model_path"]
    model_name = model_path.split("/")[-1]
    new_model_path = f"{filtered_binders_path}/{model_name}"
    # shutil.copyfile(model_path, new_model_path)

    # Generate msa
    # binder_seq=generate_fasta_sequence_from_pdb(row["model_path"])["B"]
    binder_seq = row["seq"].split("/")[-1]
    a3m_file_name = f"{msa_folder_af2}/{model_name.split('.')[0]}.a3m"
    # sample_a3m_path_af2="/home/tsatler/RFdif/ClusterProteinDesign/scripts/binder_design/output/5fmv_domain3-4/filtered_sequences/msa_inputs_af2/sample/sample.a3m"
    process_a3m_file_af2(binder_seq, sample_a3m_path_af2, a3m_file_name)

    # Update DataFrame with new values
    filtered.loc[i, "new_model_path"] = new_model_path
    filtered.loc[i, "generated_a3m_af2"] = a3m_file_name

# Check A3M files generated
msa_a3m_inputs = glob.glob(f"{msa_folder_af2}/*a3m")
print(f"There is {len(msa_a3m_inputs)} a3m files ready to be predicted with AF2!")

### Run prediction

In [ ]:
# Set the prediction options as a dictionary with booleans
prediction_options = {
    "colabfold": True,  # Set to True if you want to run ColabFold
    "rosettafold2": True,  # Set to True if you want to run RosettaFold2 (not currently used)
}

array_limit = 10
binder_analysis = "binder-second"  # if you want to perform binder analysis
msa_folder = f"output/{input}/filtered_sequences"

array_number = len(msa_a3m_inputs)
tools = " ".join(
    [tool for tool, should_run in prediction_options.items() if should_run]
)
print(tools)
bash_arguments = f"--output=/dev/null --array=0-{array_number}%{array_limit}"
script_arguments = f"{msa_folder} {filtered_binders_path} {binder_analysis} {tools} "

command = (
    f"sbatch {bash_arguments} helper_scripts/predict_binders.sh {script_arguments}"
)
print(command)

In [ ]:
subprocess.run(command, shell=True)

### Append and filter with scores

In [ ]:
# af2_dataframe_columns = ['plddt', 'pae', 'binder_plddt', 'target_plddt', 'pae_binder', 'pae_target', 'pae_int_tot', 'rmsd', 'name', 'model_id']
# rf2_dataframe_columns = ['mean_plddt', 'pae_chain0_0', 'rmsd', 'name', 'model_id', 'plddt_target', 'plddt_binder', 'plddt', 'pae']

# Open filtered dataframe
filtered = pd.read_csv(f"output/{input}/filtered_sequences/0_filtered_binders.csv")
filtered["model_id"] = (
    filtered["model_path"].str.split("/").str[-1].str.split(".").str[0]
)

# Open scores to append
af2_scores = pd.read_csv(
    f"output/{input}/filtered_sequences/filtered_binders/scores/scores_af2.csv"
)
rf2_scores = pd.read_csv(
    f"output/{input}/filtered_sequences/filtered_binders/scores/scores_rf2.csv"
)
# Fix old model_ids
af2_scores["model_id"] = af2_scores["model_id"].str.replace(r"-AF2$", "", regex=True)
rf2_scores["model_id"] = rf2_scores["model_id"].str.replace(r"-RF2$", "", regex=True)

# Columns to append
af2_column_append = ["binder_plddt", "pae_int_tot", "rmsd"]
rf2_column_append = ["plddt_binder", "pae", "rmsd"]

# Prepare columns to add
rf2_scores = rf2_scores[rf2_column_append + ["model_id"]].rename(
    columns=lambda x: f"rf_{x}" if x != "model_id" else x
)
af2_scores = af2_scores[af2_column_append + ["model_id"]].rename(
    columns=lambda x: f"af_{x}" if x != "model_id" else x
)

# Merge to filtered
filtered = filtered.merge(af2_scores, on="model_id", how="left")
filtered = filtered.merge(rf2_scores, on="model_id", how="left")
filtered.describe()

### Filter sequences

In [ ]:
filtered = filtered[
    (filtered["af_binder_plddt"] > 80)
    & (filtered["af_pae_int_tot"] < 7)
    & (filtered["af_rmsd"] < 2)
    &
    # (filtered["rf_plddt_binder"]>78)&
    # (filtered["rf_pae"]<15)&
    (filtered["rf_rmsd"] < 22)
]

filtered = filtered.sort_values(by="ddg_dsasa_100", ascending=True).drop_duplicates(
    "seq"
)
filtered.to_csv(
    f"output/{input}/filtered_sequences/1_filtered_binders.csv", index=False
)
scaffold_counts = filtered["scaffold_name"].value_counts()
total_unique_scaffolds = len(scaffold_counts)
total_scaffold_instances = scaffold_counts.sum()

print("Total unique scaffolds:", total_unique_scaffolds)
print("Total scaffold instances:", total_scaffold_instances)
print("\nScaffold counts:")
print(scaffold_counts)
filtered

# Cluster sequences

In [ ]:
filtered["seq_split"] = filtered["seq"].apply(lambda x: x.split("/")[-1])

num_clusters = 24

seqs = filtered["seq_split"].to_list()
# matrix = np.asarray([np.frombuffer(seq.encode(), dtype=np.uint8) for seq in seqs])
max_length = max(len(seq) for seq in seqs)
padded_seqs = [seq.ljust(max_length, "N") for seq in seqs]
matrix = np.asarray(
    [np.frombuffer(seq.encode(), dtype=np.uint8) for seq in padded_seqs]
)
clusterid, error, nfound = kcluster(matrix, nclusters=num_clusters)

# Apply t-SNE to the matrix to reduce the dimensionality and visualize the sequences.
tsne = TSNE(n_components=2, random_state=42)
embedded_matrix = tsne.fit_transform(matrix)

# Create a scatter plot of the embedded points and label them with cluster IDs.
plt.figure(figsize=(10, 6))
for cluster in range(num_clusters):
    cluster_points = embedded_matrix[clusterid == cluster]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f"Cluster {cluster}")

plt.title(f"t-SNE Visualization of {input} best protein sequences")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend()
# plt.savefig(f"output/{input}/filtered_sequences/tsne_binders.png")
plt.show()

# Add cluster id to dataframe
filtered["clusterid"] = clusterid
# filtered.to_csv(f"output/{input}/filtered_sequences/2_filtered_binders_clus.csv", index=False)

cluster_counts = Counter(filtered["clusterid"])

# Group the DataFrame by 'clusterid' and get unique scaffold names for each group
unique_scaffold_names = filtered.groupby("clusterid")["scaffold_name"].unique()

# Iterate over the groups and print both cluster counts and unique scaffold names
for cluster_id, scaffold_names in unique_scaffold_names.items():
    print(f"Cluster {cluster_id}: {cluster_counts[cluster_id]} sequences")
    print(f'Scaffold Names: {", ".join(scaffold_names)}')

# Calculate average cluster metrics
average_metrics_by_cluster = filtered.groupby("clusterid").mean()
# average_metrics_by_cluster.to_csv(f"output/{input}/filtered_sequences/2_cluster_average.csv", index=False)
average_metrics_by_cluster

# Output final dataframe and binders
Code below will create folders with best [12, 24, 48 ..] binders and metrics.
Make sure to change cluster ratios based on the results above!

In [ ]:
# Select number of final sequences and cluster ratios
# total_numbers=[12, 24, 48]
total_numbers = [24, 48, 96]

cluster_ratios = [0.2, 0.2, 0.2, 0.2, 0.2]
cluster_ratios = np.full(num_clusters, 1 / num_clusters)

# Function to calculate the number of sequences for each cluster based on the given ratios
def calculate_sequences_ratios(total_sequences, cluster_ratios):
    cluster_sequences = {}
    total_ratio = sum(cluster_ratios)
    remaining_sequences = total_sequences

    for i, ratio in enumerate(cluster_ratios):
        if i == len(cluster_ratios) - 1:
            cluster_sequences[i] = remaining_sequences
        else:
            sequences_for_cluster = int(total_sequences * ratio / total_ratio)
            cluster_sequences[i] = sequences_for_cluster
            remaining_sequences -= sequences_for_cluster

    return cluster_sequences

In [ ]:
for total_sequences in total_numbers:
    # Calculate the number of sequences for each cluster
    cluster_sequences = calculate_sequences_ratios(total_sequences, cluster_ratios)
    print(f"Total Sequences: {total_sequences}")
    print(cluster_sequences)

    # Create a new dataframe with the final number of sequences for each cluster
    final_sequences = []
    for cluster_id, num_sequences in cluster_sequences.items():
        cluster_data = filtered[filtered["clusterid"] == cluster_id].iloc[
            :num_sequences
        ]
        final_sequences.append(cluster_data)

    final_dataframe = pd.concat(final_sequences)

    # Save the dataframe to a CSV file
    binder_pdbs_path = f"output/{input}/filtered_sequences/final_results/binders_{total_sequences}/binder_{total_sequences}_pdbs"
    os.makedirs(binder_pdbs_path, exist_ok=True)
    filename = f"output/{input}/filtered_sequences/final_results/binders_{total_sequences}/final_binders_{total_sequences}.csv"
    final_dataframe.to_csv(filename, index=False)

    for _, row in final_dataframe.iterrows():
        pdb_path = row["model_path"]

        # filtered_binders_path=f"output/{input}/filtered_sequences/colab_rf_eval_2domain_and_binder/filtered_binders"s
        # Copy pdb in filtered folder
        # model_path=row["model_path"]
        # model_name=model_path.split("/")[-1]
        # pdb_path=f"{filtered_binders_path}/{model_name}"

        if os.path.exists(pdb_path) and os.path.isfile(pdb_path):
            pdb_filename = os.path.basename(pdb_path)
            destination_path = os.path.join(binder_pdbs_path, pdb_filename)
            shutil.copy(pdb_path, destination_path)
            # print(f"Copied {pdb_filename} to {destination_path}")
        else:
            print(f"File not found: {pdb_path}")

    print(f"DataFrame for {total_sequences} sequences saved to {filename}")